# Implementation of SELUs & Dropout-SELUs in NumPy
This looks pretty neat. 
They can prove that when you slightly modify the ELU activation,
your average unit activation goes towards zero mean/unit variance (if the network is deep enough). 
If they're right, this might make batch norm obsolete, which would be a huge bon to training speeds! 

The experiments look convincing, so apparently it even beats BN+ReLU in accuracy... though 

I wish they would've shown the resulting distributions of activations after training. 

But assuming their fixed point proof is true, it will. 

Still, still would've been nice if they'd shown it -- maybe they ran out of space in their appendix ;)

Weirdly, the exact ELU modification they proposed isn't stated explicitly in the paper! 

For those wondering, it can be found in the available sourcecode, and looks like this:

In [30]:
# An extra explaination from Reddit
# # Thanks, I will double check the analytical solution. For the numerical one, could you please explain why running the following code results in a value close to 1 rather than 0?
# du = 0.001
# u_old = np.mean(selu(np.random.normal(0,    1, 100000000)))
# u_new = np.mean(selu(np.random.normal(0+du, 1, 100000000)))
# # print (u_new-u_old) / du
# print(u_old, u_new)
# # Now I see your problem: 
# #     You do not consider the effect of the weights. 
# #     From one layer to the next, we have two influences: 
# #         (1) multiplication with weights and 
# #         (2) applying the SELU. 
# #         (1) has a centering and symmetrising effect (draws mean towards zero) and 
# #         (2) has a variance stabilizing effect (draws variance towards 1). 

# #         That is why we use the variables \mu&\omega and \nu&\tau to analyze the both effects.
# # Oh yes, thats true, zero mean weights completely kill the mean. Thanks!

# Tensorflow implementation
import numpy as np

def selu(x):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))

In [31]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x = selu(x=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.169643980483 0.19326595588 0.831494203398 1.17907301442
-0.195905368533 0.187288445337 0.813347344359 1.16342946743
-0.165612815069 0.256356293921 0.814312012191 1.19610052215
-0.180493963366 0.21574595991 0.807861023787 1.18077772604
-0.1572638478 0.157882898432 0.831537691138 1.19411239372
-0.170677280441 0.238829762852 0.819557386879 1.18223208257
-0.19756382144 0.223737118683 0.844352863237 1.1677981833
-0.152764904203 0.182116135091 0.83884410518 1.20924303205
-0.232590338728 0.187804600082 0.809210210667 1.18526226435
-0.195643972324 0.171561714244 0.832400619265 1.18972806518
-0.211125263462 0.207497349536 0.815235769375 1.14424488443
-0.179760875045 0.194749334604 0.802945690182 1.15404773795
-0.171757947632 0.20891807862 0.766485697323 1.18413733708
-0.180197385493 0.190346664807 0.820735061854 1.18700959668
-0.156624103796 0.20947264287 0.838359239266 1.16184159831
-0.189810181753 0.170410272796 0.844955944581 1.1752121995
-0.170784407303 0.194969660057 0.798726494868 1.21

In [32]:
# My NumPy implemetation of Normal dropout for ReLU
def dropout_forward(X, p_dropout):
    u = np.random.binomial(1, p_dropout, size=X.shape) / p_dropout
    out = X * u
    cache = u
    return out, cache

def dropout_backward(dout, cache):
    dX = dout * cache
    return dX

In [33]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x = selu(x)
    x, _ = dropout_forward(p_dropout=0.8, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.251072013541 0.209376587988 0.91172223785 1.33068686397
-0.27614508784 0.288469922165 0.968164422361 1.47335482914
-0.214378498633 0.302353397343 1.04634005926 1.70487722628
-0.215165648961 0.350261002442 1.09541807431 1.69191290108
-0.191359108966 0.410521963591 1.14341834698 1.8230372023
-0.199306717222 0.528614010651 1.17955843745 1.81181994637
-0.162545509646 0.537175309387 1.17090324424 1.91887732184
-0.15792265875 0.417386601627 1.22381079532 1.90862400086
-0.194858398318 0.471407967953 1.2094845386 1.95818537782
-0.18064306425 0.47771441999 1.24270997738 2.03678454329
-0.20996212582 0.58320725105 1.20960018578 2.07392270081
-0.092738860604 0.646332186735 1.23685342565 2.13697041661
-0.18380841256 0.572740835567 1.23920963383 2.15900680026
-0.100584538208 0.501067627891 1.25326889614 2.10984398015
-0.150609705106 0.482804318562 1.22356591219 2.17064366355
-0.139344365796 0.625975347271 1.24359615778 2.03614583347
-0.259280647441 0.67430472889 1.19352014915 2.22979972235
-0.165

In [34]:
# Tensorflow implementation on github
def dropout_selu(x, rate, alpha= -1.7580993408473766, fixedPointMean=0.0, fixedPointVar=1.0, 
                 noise_shape=None, seed=None, name=None, training=False):
    """Dropout to a value with rescaling."""

    def dropout_selu_impl(x, rate, alpha, noise_shape, seed, name):
        keep_prob = 1.0 - rate
        x = ops.convert_to_tensor(x, name="x")
        if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
            raise ValueError("keep_prob must be a scalar tensor or a float in the "
                                             "range (0, 1], got %g" % keep_prob)
        keep_prob = ops.convert_to_tensor(keep_prob, dtype=x.dtype, name="keep_prob")
        keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

        alpha = ops.convert_to_tensor(alpha, dtype=x.dtype, name="alpha")
        keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

        if tensor_util.constant_value(keep_prob) == 1:
            return x

        noise_shape = noise_shape if noise_shape is not None else array_ops.shape(x)
        random_tensor = keep_prob
        random_tensor += random_ops.random_uniform(noise_shape, seed=seed, dtype=x.dtype)
        binary_tensor = math_ops.floor(random_tensor)
        ret = x * binary_tensor + alpha * (1-binary_tensor)

        a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))

        b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
        ret = a * ret + b
        ret.set_shape(x.get_shape())
        return ret

    with ops.name_scope(name, "dropout", [x]) as name:
        return utils.smart_cond(training,
            lambda: dropout_selu_impl(x, rate, alpha, noise_shape, seed, name),
            lambda: array_ops.identity(x))

In [35]:
def elu_fwd(X):
    alpha = 1.0
    scale = 1.0
    #     return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))
    X_pos = np.maximum(0.0, X) # ReLU
    X_neg = np.minimum(X, 0.0) # otherwise: if X<=0, Exp Leaky ReLU
    X_neg_exp = alpha * (np.exp(X_neg)-1) # a: slope, a>=0
    out = scale * (X_pos + X_neg_exp)
    cache = (scale, alpha, X) # mean=0, std=1
    return out, cache

def elu_bwd(dout, cache):
    scale, alpha, X = cache # mean=0, std=1
    dout = dout * scale
    dX_neg = dout.copy()
    dX_neg[X>0] = 0
    X_neg = np.minimum(X, 0) # otherwise: if X<=0, Exp Leaky ReLU
    dX_neg = dX_neg * alpha * np.exp(X_neg) # derivative of abs(np.exp(X_neg)-1) # a: slope, a>=0
    dX_pos = dout.copy()
    dX_pos[X<0] = 0
    dX_pos = dX_pos * 1
    dX = dX_neg + dX_pos
    return dX

In [36]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x, _ = elu_fwd(X=x)
    x, _ = dropout_forward(p_dropout=0.95, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.00617050592252 0.337673784589 0.646991767381 0.962698565859
-0.0575137185421 0.255565877841 0.542622466441 0.854510964778
-0.0230847296611 0.199563161172 0.445756221729 0.727346459383
-0.016513908502 0.136585717373 0.388690840169 0.6546448826
-0.0326313282382 0.153107850779 0.333549029594 0.572333028999
-0.0422632935911 0.137046160828 0.307162464527 0.546718161965
-0.0285822914336 0.12916218473 0.273611287149 0.515238746334
-0.0478919735396 0.118872121176 0.245816897573 0.502284015533
-0.0458842998415 0.092463329908 0.211156449503 0.47229103747
-0.00918631706758 0.084913351153 0.199205721295 0.449707867937
-0.0228903538636 0.0808967569073 0.186963659871 0.38590401974
-0.0302475644433 0.0798865404621 0.178715249854 0.367585993646
-0.0306024780895 0.0532624298075 0.162414124151 0.340216091126
-0.0212268755842 0.0561615458685 0.159430684408 0.342516259688
-0.0222513430424 0.0557010615243 0.147176968751 0.325957270058
-0.0309229442399 0.0475138824285 0.145784383225 0.28792104075
-0.0262

In [37]:
def selu_fwd(X):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    #     return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))
    X_pos = np.maximum(0.0, X) # ReLU
    X_neg = np.minimum(X, 0.0) # otherwise: if X<=0, Exp Leaky ReLU
    X_neg_exp = alpha * (np.exp(X_neg)-1) # a: slope, a>=0
    out = scale * (X_pos + X_neg_exp)
    cache = (scale, alpha, X) # mean=0, std=1
    return out, cache

def selu_bwd(dout, cache):
    scale, alpha, X = cache # mean=0, std=1
    dout = dout * scale
    dX_neg = dout.copy()
    dX_neg[X>0] = 0
    X_neg = np.minimum(X, 0) # otherwise: if X<=0, Exp Leaky ReLU
    dX_neg = dX_neg * alpha * np.exp(X_neg) # derivative of abs(np.exp(X_neg)-1) # a: slope, a>=0
    dX_pos = dout.copy()
    dX_pos[X<0] = 0
    dX_pos = dX_pos * 1
    dX = dX_neg + dX_pos
    return dX

# def dropout_selu_forward(X, p_dropout):
def dropout_selu_forward(X, keep_prob):
    alpha= -1.7580993408473766
    fixedPointMean=0.0
    fixedPointVar=1.0

    u = np.random.binomial(1, keep_prob, size=X.shape) / keep_prob
    out = X * u + alpha * (1-u)

    #     a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))
    a = np.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * (alpha-fixedPointMean)**2 + fixedPointVar)))
    b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
    out = a * out + b
    cache = a, u
    return out, cache

def dropout_selu_backward(dout, cache):
    a, u = cache
    dout = dout * a
    dX = dout * u
    return dX

In [38]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x, cache = selu_fwd(x)
    x, _ = dropout_selu_forward(keep_prob=0.95, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.128076676934 0.345373998275 0.896616335816 1.23852215989
-0.131370484455 0.317562586576 0.88544207764 1.36043184665
-0.149992044707 0.301410324278 0.93351947205 1.29715189353
-0.113140599252 0.336050568982 0.888520950881 1.34689270297
-0.121992763956 0.329679584586 0.919620670954 1.36270007557
-0.121133497655 0.314879313582 0.964087664337 1.34620887674
-0.0847146343084 0.365416689223 0.97900041256 1.37083523972
-0.0915272851586 0.300225203273 0.972339324631 1.37597180954
-0.104276864299 0.357964795285 0.991599333995 1.3957291022
-0.0943609404869 0.310337867312 0.993450035038 1.37346641576
-0.0876511594656 0.376730864096 0.985113284654 1.45074599094
-0.0853973489834 0.335766057655 1.00027288442 1.4329469673
-0.0738033660733 0.373493385626 1.00256920071 1.45065317169
-0.0975552843196 0.435007371725 0.970736819116 1.4555096332
-0.110054128154 0.404458233626 0.967042990893 1.43269307518
-0.112845667359 0.39379534816 0.997164004504 1.47155570554
-0.110709560509 0.319015684186 0.937776501

# Discussion & wrapup
According to this, even after a 100 layers, mean neuron activations stay fairly close to mean 0 / variance 1 
(even the most extreme means/variances are only off by 0.2).

Sepp Hochreiter is amazing: LSTM, meta-learning, SNNN. 

I think he has already done a much larger contribution to science than some self-proclaimed pioneers of DL 
who spend more time on social networks than actually doing any good research.